In [ ]:
import altair as alt
import polars as pl
from pycountry import countries

In [ ]:
%reload_ext sql

In [ ]:
%sql duckdb:////data/duckdb/foursquare.duckdb

In [ ]:
%sql df << SELECT country AS cc, COUNT(*) AS count FROM fs GROUP BY ALL ORDER BY count DESC

In [ ]:
def get_country_name(code):
    try:
        name = countries.get(alpha_2=code).name
        return f"{name} ({code})"
    except KeyError:
        return code

In [ ]:
df = df.select(
    pl.col("*"),
    pl.col("cc")
    .map_elements(get_country_name, return_dtype=pl.String)
    .alias("country"),
)

In [ ]:
chart = (
    alt.Chart(df.head(40), title="Foursquare locations by country")
    .mark_bar(tooltip=True)
    .encode(
        alt.X("count", title="Number of locations"),
        alt.Y("country", sort="-x", title="Country"),
        alt.Color("country", sort="-x"),
    )
    .properties(width=400)
)

alt.JupyterChart(chart)

In [ ]:
%sql SELECT COUNT(*) FROM fs

In [ ]:
%sql pike_df << SELECT name, address, postcode, latitude, longitude FROM fs WHERE postcode LIKE 'GL6 0%'

In [ ]:
pike_chart = (
    alt.Chart(pike_df, title="Foursquare locations")
    .mark_circle(tooltip=True)
    .encode(
        alt.Latitude("latitude:Q"),
        alt.Longitude("longitude:Q"),
        alt.Tooltip("name"),
    )
    .project("equalEarth", center=(-2.225, 51.69), scale=1200000)
    .properties(width=800, height=800)
)

alt.JupyterChart(pike_chart)